In [2]:
import pandas as pd

In [3]:
book_score = pd.read_csv('Book_score.csv',encoding="gbk")

In [4]:
print(book_score.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209212 entries, 0 to 209211
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     209212 non-null  int64 
 1   book_id     209212 non-null  int64 
 2   book_score  209212 non-null  int64 
 3   time        209212 non-null  object
 4   tag         209212 non-null  object
dtypes: int64(3), object(2)
memory usage: 8.0+ MB
None


In [5]:
user_set = set(book_score['user_id'])
user_size = len(user_set)
item_set = set(book_score['book_id'])
item_size = len(item_set)

print(user_size)
print(item_size)


2337
1000


In [6]:
user_list = list(user_set)
item_list = list(item_set)
user_map = {}
user_map_r = {}
item_map = {}
item_map_r = {}
for j in range(len(user_list)):
    user_map[str(j)] = user_list[j]
    user_map_r[str(user_list[j])] = j
for k in range(len(item_list)):
    item_map[str(k)] = item_list[k]
    item_map_r[str(item_list[k])] = k
print(len(user_map))
print(len(user_map_r))
print(len(item_map))
print(len(item_map_r))

2337
2337
1000
1000


In [7]:
num_neighbor = 10

In [8]:
import numpy as np

In [9]:
book_score_backup = book_score

In [10]:
for userId in user_list:
    book_score.loc[book_score['user_id']==userId, 'user_id'] = user_map_r[str(userId)]
for itemId in item_list:
    book_score.loc[book_score['book_id']==itemId, 'book_id'] = item_map_r[str(itemId)]

In [11]:
dataList = book_score.groupby(by='user_id').agg({'book_id':list})
dataList['user_id'] = dataList.index
dataList.reset_index(drop=True)

,book_id,user_id
0,"[219, 818, 808, 366, 882, 127, 708, 329, 243, ...",0
1,"[704, 573, 843, 492, 384, 896, 647, 130, 700, ...",1
2,"[53, 202, 766, 242, 508, 490, 92, 168, 829, 41...",2
3,"[630, 454, 294, 217, 430, 68, 526, 327, 160, 7...",3
4,"[637, 967, 353, 826, 398, 388, 285, 260, 848, ...",4
...,...,...
2332,"[804, 233, 283, 708, 524, 863, 896, 130, 533, ...",2332
2333,"[801, 863, 582, 678, 127, 700, 279, 543, 694, ...",2333
2334,"[241, 896, 130, 533, 613, 148, 567, 964, 80, 7...",2334
2335,"[981, 871, 1, 340, 979, 404, 35, 641, 535, 242...",2335


In [12]:
print(dataList)

                                                   book_id  user_id
user_id                                                            
0        [219, 818, 808, 366, 882, 127, 708, 329, 243, ...        0
1        [704, 573, 843, 492, 384, 896, 647, 130, 700, ...        1
2        [53, 202, 766, 242, 508, 490, 92, 168, 829, 41...        2
3        [630, 454, 294, 217, 430, 68, 526, 327, 160, 7...        3
4        [637, 967, 353, 826, 398, 388, 285, 260, 848, ...        4
...                                                    ...      ...
2332     [804, 233, 283, 708, 524, 863, 896, 130, 533, ...     2332
2333     [801, 863, 582, 678, 127, 700, 279, 543, 694, ...     2333
2334     [241, 896, 130, 533, 613, 148, 567, 964, 80, 7...     2334
2335     [981, 871, 1, 340, 979, 404, 35, 641, 535, 242...     2335
2336     [583, 587, 751, 824, 707, 172, 353, 116, 483, ...     2336

[2337 rows x 2 columns]


In [13]:
print(dataList)

                                                   book_id  user_id
user_id                                                            
0        [219, 818, 808, 366, 882, 127, 708, 329, 243, ...        0
1        [704, 573, 843, 492, 384, 896, 647, 130, 700, ...        1
2        [53, 202, 766, 242, 508, 490, 92, 168, 829, 41...        2
3        [630, 454, 294, 217, 430, 68, 526, 327, 160, 7...        3
4        [637, 967, 353, 826, 398, 388, 285, 260, 848, ...        4
...                                                    ...      ...
2332     [804, 233, 283, 708, 524, 863, 896, 130, 533, ...     2332
2333     [801, 863, 582, 678, 127, 700, 279, 543, 694, ...     2333
2334     [241, 896, 130, 533, 613, 148, 567, 964, 80, 7...     2334
2335     [981, 871, 1, 340, 979, 404, 35, 641, 535, 242...     2335
2336     [583, 587, 751, 824, 707, 172, 353, 116, 483, ...     2336

[2337 rows x 2 columns]


In [14]:
negative = dict()  #做负采样

In [15]:
config = {
    'batch_size': 128,
    'epochs': 10,
    'weight_decay': 1e-6,
    'num_classes': 2,
    'lr': 2e-5,
    'negative_sum':200
}

In [16]:
book_list = list(item_set)
for i in range(len(book_list)):
    book_list[i] = item_map_r[str(book_list[i])]
print(book_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [64]:
for userId in dataList['user_id']:
    negatives = list()
    while len(negatives) < config['negative_sum']:
        randId = np.random.randint(0, 1000)
        book_id = book_list[randId]
        if book_id not in dataList.loc[userId].book_id:
            negatives.append(book_id)
    str_user_id = str(userId)
    negative[str_user_id] = negatives

In [17]:
book_score = book_score.explode('book_id').reset_index(drop=True)
book_score['label'] = 1
book_score['user_id'].max()
#book_score['user_id'].max()
#book_score.head()

2336

In [18]:
book_score_filter = book_score
book_score_filter = book_score_filter.drop('time', axis=1)
book_score_filter = book_score_filter.drop('tag', axis=1)
book_score_filter.head()

,user_id,book_id,book_score,label
0,1684,668,0,1
1,1684,128,0,1
2,1684,976,5,1
3,1684,786,5,1
4,1684,480,5,1


In [67]:
negative_df = pd.DataFrame.from_dict(negative, orient='index')
negative_df['user_id'] = negative_df.index
negative_df['book_id'] = negative_df.apply(lambda x: [x[i] for i in range(config['negative_sum'])], axis=1)
negative_df = negative_df[['user_id', 'book_id']]
negative_df = negative_df.explode('book_id').reset_index(drop=True)
negative_df['label'] = 0
negative_df.insert(loc=2,column='book_score',value=0)
print(negative_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467400 entries, 0 to 467399
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     467400 non-null  object
 1   book_id     467400 non-null  object
 2   book_score  467400 non-null  int64 
 3   label       467400 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 14.3+ MB
None


In [68]:
negative_df.head()

,user_id,book_id,book_score,label
0,0,114,0,0
1,0,151,0,0
2,0,698,0,0
3,0,607,0,0
4,0,803,0,0


In [69]:
ave_rate = dict()
for bookId in book_list:
    ave_rate[str(bookId)] = round(book_score_filter[book_score_filter['book_id']==bookId]['book_score'].mean(),2)
print(ave_rate)

{'0': 2.81, '1': 2.91, '2': 2.42, '3': 3.31, '4': 3.32, '5': 3.34, '6': 2.74, '7': 1.86, '8': 3.01, '9': 3.47, '10': 2.91, '11': 3.69, '12': 3.13, '13': 2.94, '14': 2.19, '15': 2.87, '16': 3.08, '17': 3.5, '18': 3.18, '19': 3.36, '20': 3.5, '21': 3.56, '22': 3.57, '23': 4.32, '24': 3.45, '25': 3.35, '26': 3.2, '27': 2.36, '28': 3.39, '29': 3.17, '30': 2.65, '31': 3.47, '32': 3.51, '33': 3.09, '34': 3.56, '35': 3.05, '36': 2.74, '37': 2.62, '38': 3.27, '39': 3.27, '40': 2.6, '41': 2.98, '42': 2.5, '43': 3.33, '44': 3.19, '45': 2.84, '46': 3.51, '47': 3.31, '48': 2.62, '49': 3.04, '50': 3.03, '51': 2.94, '52': 3.8, '53': 3.51, '54': 2.79, '55': 3.42, '56': 3.44, '57': 3.29, '58': 3.35, '59': 2.82, '60': 2.53, '61': 2.67, '62': 3.29, '63': 2.85, '64': 3.61, '65': 2.49, '66': 3.64, '67': 2.81, '68': 2.82, '69': 3.12, '70': 3.56, '71': 3.63, '72': 3.03, '73': 3.0, '74': 2.45, '75': 3.59, '76': 2.46, '77': 2.27, '78': 2.85, '79': 3.03, '80': 3.11, '81': 3.24, '82': 2.5, '83': 2.88, '84': 3.1

In [70]:
for bookId in book_list:
    negative_df.loc[negative_df['book_id']==bookId,'book_score'] = ave_rate[str(bookId)]
negative_df.head()

,user_id,book_id,book_score,label
0,0,114,3.43,0
1,0,151,3.02,0
2,0,698,2.72,0
3,0,607,2.51,0
4,0,803,2.65,0


In [59]:
from sklearn.model_selection import train_test_split
#data = pd.concat([book_score_filter, negative_df]).astype(np.int32)
data = book_score_filter
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:,0:2], data.iloc[:, 2], test_size=0.2, random_state=100)
x_test.shape
train_set = pd.concat([x_train,y_train],axis=1)
test_set = pd.concat([x_test,y_test],axis=1)
for userId in range(len(user_list)):
    train_set.loc[train_set['user_id']==userId, 'user_id'] = user_map[str(userId)]
    test_set.loc[test_set['user_id']==userId, 'user_id'] = user_map[str(userId)]
for itemId in range(len(item_list)):
    train_set.loc[train_set['book_id']==itemId, 'book_id'] = item_map[str(itemId)]
    test_set.loc[test_set['book_id']==itemId, 'book_id'] = item_map[str(itemId)]
print(train_set)
#train_set.to_csv('./Train_set.csv',sep=',', header=True,index=False)
#test_set.to_csv('./Test_set.csv',sep=',', header=True,index=False)

        user_id  book_id  book_score
178353  1995403  1080309           4
1583    1943109  1057244           0
120566  1529713  2161892           4
173591  1873494  1844629           5
124967  2169835  1331571           0
...         ...      ...         ...
194442  1322968  1029159           5
65615   2143614  2149655           4
77655   1491428  1046385           0
56088   4540110  1039487           3
38408   2735218  5317075           4

[167369 rows x 3 columns]


In [20]:
from torch.utils.data import DataLoader, Dataset
import torch
class BookDataset(Dataset):
    def __init__(self, x, y):
        super(BookDataset, self).__init__()
        self.x = x
        self.y = y
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    def __len__(self):
        return len(self.x)
train_dataset = BookDataset(torch.tensor(x_train.values), torch.tensor(y_train.values))
test_dataset = BookDataset(torch.tensor(x_test.values), torch.tensor(y_test.values))
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])
print(len(train_loader))

1308


In [21]:
import torch
import torch.nn as nn
class NeuralCF(nn.Module):
    def __init__(self, userIds, itemIds):
        super(NeuralCF, self).__init__()
        self.userEmb = nn.Embedding(userIds, 64)
        self.bookEmb = nn.Embedding(itemIds, 64)
        self.sequential = nn.Sequential(
            nn.Linear(128, 96),
            nn.ReLU(),
            nn.Linear(96, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def _init_weight_(self):
        nn.init.normal_(self.userEmb.weight,std=0.01)
        nn.init.normal_(self.bookEmb.weight,std=0.01)
    def forward(self, x):
        left = torch.LongTensor(x[:,0].numpy())
        right = torch.LongTensor(x[:,1].numpy())
        left = self.userEmb(left)
        right = self.bookEmb(right)
        x = torch.cat([left, right], dim=1)
        x = self.sequential(x)
        x = torch.squeeze(x)
        return x

In [22]:
def DCG(rank, rate):
    rate = rate / 5
    return (np.power(2, rate) - 1) / np.log2(rank + 1)

In [176]:

from sklearn.metrics import accuracy_score
# 定义损失函数
def criterion(outputs, labels):
    return nn.MSELoss()(outputs, labels)
# 计算准确率
def getNDCG(outputs, labels):
    output = {}
    label = {}
    for i in range(len(outputs)):
        output[str(i)] = outputs[i]
        label[str(i)] = labels[i]
    output = sorted(output.items(), key=lambda x: x[1], reverse=True)
    label = sorted(label.items(), key=lambda x: x[1], reverse=True)
    #list
    dcgs_output = []
    idcgs_label = []
    for i in range(len(output)):
        for items in label:
            if items[0] == output[i][0]:
                dcgs_output.append(DCG(i+1,items[1]))
            if items[0] == label[i][0]:
                idcgs_label.append(DCG(i+1,items[1]))
    return dcgs_output,idcgs_label


In [24]:
from tqdm.auto import tqdm
import gc 
def train_one_epoch(epoch, model, dataLoader, optimizer):
    model.train()
    steps = len(dataLoader)
    bar = tqdm(enumerate(dataLoader), total=len(dataLoader))
    dataset_size = 0
    running_loss = 0
    for step, (x, y) in bar:
        batch_size = x.shape[0]
        outputs = model(x)
        y = torch.FloatTensor(y.numpy())
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size
        
        bar.set_description(f'Epoch: [{epoch}/{10}]')
        bar.set_postfix(Epoch=epoch, Train_loss=epoch_loss)
    gc.collect()
    return running_loss

In [25]:
def valid_one_epoch(epoch, model, dataLoader):
    model.eval()
    bar = tqdm(enumerate(dataLoader), total=len(dataLoader))
    DCG_output = []
    IDCG_label = []
    DCG_sum = 0
    IDCG_sum = 0
    item = 0
    for step, (x, y) in bar:
        item += 1
        batch_size = x.shape[0]
        outputs = model(x)
        DCG_output, IDCG_label = getNDCG(outputs, y)
        DCG_sum += sum(DCG_output)
        IDCG_sum += sum(IDCG_label)
        bar.set_description(f'Epoch: [{epoch}/{10}]')
        bar.set_postfix({'NDCG' : '{0:1.5f}'.format(DCG_sum/IDCG_sum)})
    gc.collect()
    return DCG_sum/IDCG_sum

In [26]:
model = NeuralCF(2337,1000)
total_NDCG = 0
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
for epoch in range(1, config['epochs'] + 1):
    train_one_epoch(epoch, model, train_loader, optimizer)
    total_NDCG = valid_one_epoch(epoch, model, test_loader)
print(f'total NDCG:{total_NDCG}')


Epoch: [10/10]: 100%|██████████| 327/327 [00:08<00:00, 36.36it/s, NDCG=0.91482]


total NDCG:0.9148208821959609


In [27]:
#TODO:Rank List for every users
def ComputeALlRate(model, dataLoader):
    bar = tqdm(enumerate(dataLoader), total=len(dataLoader))
    for step, (x, y) in bar:
        batch_size = x.shape[0]
        outputs = model(x)
    return outputs

In [29]:
print(len(test_dataset))

41843


In [31]:
test_loader1 = DataLoader(test_dataset, batch_size=len(test_dataset))

In [51]:
rank_list = ComputeALlRate(model, test_loader1)

100%|██████████| 1/1 [00:00<00:00,  1.89it/s]


In [52]:
print(len(rank_list))
print(rank_list)
rank_list = rank_list.detach().numpy().tolist()

41843
tensor([2.3720, 3.3161, 1.9180,  ..., 3.4755, 3.5567, 2.3273],
       grad_fn=<SqueezeBackward0>)


In [111]:
#print(x_test)
predicted = x_test.copy()
#print(predicted)
predicted.insert(loc=len(predicted.columns), column='predicted_score',value=rank_list)
print(predicted)
print(x_test)

        user_id  book_id  predicted_score
184937     1522      919         2.372016
136250     2181      225         3.316051
86770       475      199         1.917972
66072      2032      856         3.016251
138952      457      646         3.516768
...         ...      ...              ...
64196       547      239         3.026670
98166      1743      340         3.492137
123254      240      189         3.475492
125395     1698       58         3.556684
105649     2031      362         2.327331

[41843 rows x 3 columns]
        user_id  book_id
184937     1522      919
136250     2181      225
86770       475      199
66072      2032      856
138952      457      646
...         ...      ...
64196       547      239
98166      1743      340
123254      240      189
125395     1698       58
105649     2031      362

[41843 rows x 2 columns]


In [112]:
for userId in range(len(user_list)):
    predicted.loc[predicted['user_id']==userId, 'user_id'] = user_map[str(userId)]
for itemId in range(len(item_list)):
    predicted.loc[predicted['book_id']==itemId, 'book_id'] = item_map[str(itemId)]

In [113]:
predict = predicted.groupby(by='user_id').agg({'book_id':list,'predicted_score':list})
predict['user_id'] = predict.index
predict.reset_index(drop=True)
print(predict)

                                                    book_id  \
user_id                                                       
1000147   [1054685, 2257188, 1019485, 2243692, 1007305, ...   
1000152   [1084336, 1008145, 2036343, 1865891, 4886245, ...   
1000176   [1024197, 1003000, 1088022, 1291355, 1020670, ...   
1000182   [1089515, 1440589, 4238362, 1205054, 1044203, ...   
1000288   [1079440, 1024217, 1059419, 1389960, 1002299, ...   
...                                                     ...   
44827605  [1085860, 1903968, 2057285, 2133074, 2265312, ...   
44941614  [1039487, 2265312, 3268399, 3646172, 1927902, ...   
45337884  [1475048, 2359003, 1469051, 1125186, 1045818, ...   
47513621  [1054889, 1057244, 1087145, 1026425, 1082154, ...   
48943819  [1026411, 1068308, 1255625, 1056461, 1044547, ...   

                                            predicted_score   user_id  
user_id                                                                
1000147   [3.324554920196533, 3.2390

In [92]:
predict

,book_id,predicted_score,user_id
user_id,,,
1000147,"[1054685, 2257188, 1019485, 2243692, 1007305, ...","[3.324554920196533, 3.2390854358673096, 3.1507...",1000147
1000152,"[1084336, 1008145, 2036343, 1865891, 4886245, ...","[2.8716228008270264, 3.0487265586853027, 3.158...",1000152
1000176,"[1024197, 1003000, 1088022, 1291355, 1020670, ...","[2.591280221939087, 3.036618709564209, 2.65478...",1000176
1000182,"[1089515, 1440589, 4238362, 1205054, 1044203, ...","[3.3711581230163574, 3.3933937549591064, 3.648...",1000182
1000288,"[1079440, 1024217, 1059419, 1389960, 1002299, ...","[2.802184581756592, 2.6649668216705322, 2.9565...",1000288
...,...,...,...
44827605,"[1085860, 1903968, 2057285, 2133074, 2265312, ...","[3.013427257537842, 2.801945924758911, 2.45187...",44827605
44941614,"[1039487, 2265312, 3268399, 3646172, 1927902, ...","[2.6421990394592285, 2.8229875564575195, 2.558...",44941614
45337884,"[1475048, 2359003, 1469051, 1125186, 1045818, ...","[2.9149985313415527, 2.439664840698242, 3.1367...",45337884


In [93]:
predict.to_csv('./predict.csv',sep=',',index=False)
#这里打印总体的预测评分

In [184]:
print(len(predict))
#predict

2337


In [163]:
#去除book_id的个数少于10个的测试数据，计算NDCG@10
ls = []
j = -1
for i in range(len(predict['book_id'])):
    j += 1
    if len(predict.iloc[i,0]) < 10:
        ls.append(predict.iloc[i,2])
#print(ls)
predict_filter = predict.drop(ls,axis=0)
#print(ls)

[1000147, 1000375, 1000574, 1001406, 1002682, 1003232, 1003254, 1005080, 1009141, 1009530, 1010923, 1011382, 1012762, 1012873, 1016027, 1016744, 1017574, 1018568, 1018650, 1019323, 1019679, 1021243, 1022218, 1024094, 1024221, 1024314, 1024738, 1025112, 1025730, 1027565, 1029880, 1033056, 1033433, 1033510, 1035183, 1035937, 1038939, 1041534, 1044168, 1046680, 1050022, 1050739, 1052753, 1053331, 1053724, 1054495, 1056855, 1058926, 1066775, 1068383, 1069909, 1074758, 1076732, 1078081, 1078948, 1079021, 1079459, 1082828, 1086549, 1086757, 1086899, 1087188, 1089678, 1090723, 1091637, 1094936, 1095117, 1096021, 1096314, 1097027, 1099570, 1104820, 1107947, 1108215, 1108491, 1112804, 1113520, 1113664, 1113917, 1117233, 1119534, 1120366, 1125404, 1128258, 1129320, 1130324, 1132726, 1136683, 1137577, 1137624, 1140854, 1141690, 1145451, 1145553, 1145845, 1146587, 1154036, 1155235, 1155882, 1156595, 1157543, 1159916, 1162749, 1167080, 1174025, 1174955, 1182392, 1187075, 1189439, 1191409, 1191695, 

In [164]:
predict_filter

,book_id,predicted_score,user_id
user_id,,,
1000152,"[1084336, 1008145, 2036343, 1865891, 4886245, ...","[2.8716228008270264, 3.0487265586853027, 3.158...",1000152
1000176,"[1024197, 1003000, 1088022, 1291355, 1020670, ...","[2.591280221939087, 3.036618709564209, 2.65478...",1000176
1000182,"[1089515, 1440589, 4238362, 1205054, 1044203, ...","[3.3711581230163574, 3.3933937549591064, 3.648...",1000182
1000288,"[1079440, 1024217, 1059419, 1389960, 1002299, ...","[2.802184581756592, 2.6649668216705322, 2.9565...",1000288
1000430,"[1057601, 1024197, 4886245, 1051440, 1061563, ...","[2.653614044189453, 3.0408377647399902, 3.2003...",1000430
...,...,...,...
44827605,"[1085860, 1903968, 2057285, 2133074, 2265312, ...","[3.013427257537842, 2.801945924758911, 2.45187...",44827605
44941614,"[1039487, 2265312, 3268399, 3646172, 1927902, ...","[2.6421990394592285, 2.8229875564575195, 2.558...",44941614
45337884,"[1475048, 2359003, 1469051, 1125186, 1045818, ...","[2.9149985313415527, 2.439664840698242, 3.1367...",45337884


In [131]:
predict_filter.to_csv('./predict_filter.csv',sep=',',index=False)

In [145]:
dict = {}
rec = predict_filter.copy()
for i in range(len(predict_filter['user_id'])):
    dict[str(predict_filter.iloc[i,2])] = {}
    for j in range(len(predict_filter.iloc[i, 0])):
        dict[str(predict_filter.iloc[i,2])][str(predict_filter.iloc[i,0][j])] = predict_filter.iloc[i,1][j]
dict

{'1000152': {'1084336': 2.8716228008270264,
  '1008145': 3.0487265586853027,
  '2036343': 3.1580820083618164,
  '1865891': 2.8148856163024902,
  '4886245': 3.12142276763916,
  '1389960': 3.3268632888793945,
  '1087145': 2.8949851989746094,
  '1344548': 3.020129442214966,
  '1016523': 2.5763304233551025,
  '1316648': 3.392697811126709,
  '1023045': 3.293363571166992,
  '1358413': 2.6283226013183594,
  '3615061': 3.003953456878662,
  '1063624': 2.5280208587646484,
  '1021847': 3.085355520248413,
  '1020961': 3.055985927581787,
  '2257188': 2.9341306686401367,
  '2034893': 3.164803981781006,
  '1938591': 2.5065810680389404,
  '3315879': 2.810852289199829,
  '1291355': 2.833336353302002,
  '1041616': 2.8273820877075195,
  '4741216': 2.930708408355713,
  '1034282': 2.6780357360839844,
  '2361768': 3.1452674865722656,
  '1151876': 2.7033915519714355,
  '1394364': 3.4075043201446533},
 '1000176': {'1024197': 2.591280221939087,
  '1003000': 3.036618709564209,
  '1088022': 2.6547837257385254,
 

In [146]:
for i in dict.items():
    sorted_item = sorted(i[1].items(), key=lambda x: x[1], reverse=True)
    dict[i[0]] = sorted_item[0:10]
    print(len(dict[i[0]]))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

In [183]:
#print(dict)

In [165]:
predict_filter

,book_id,predicted_score,user_id
user_id,,,
1000152,"[1084336, 1008145, 2036343, 1865891, 4886245, ...","[2.8716228008270264, 3.0487265586853027, 3.158...",1000152
1000176,"[1024197, 1003000, 1088022, 1291355, 1020670, ...","[2.591280221939087, 3.036618709564209, 2.65478...",1000176
1000182,"[1089515, 1440589, 4238362, 1205054, 1044203, ...","[3.3711581230163574, 3.3933937549591064, 3.648...",1000182
1000288,"[1079440, 1024217, 1059419, 1389960, 1002299, ...","[2.802184581756592, 2.6649668216705322, 2.9565...",1000288
1000430,"[1057601, 1024197, 4886245, 1051440, 1061563, ...","[2.653614044189453, 3.0408377647399902, 3.2003...",1000430
...,...,...,...
44827605,"[1085860, 1903968, 2057285, 2133074, 2265312, ...","[3.013427257537842, 2.801945924758911, 2.45187...",44827605
44941614,"[1039487, 2265312, 3268399, 3646172, 1927902, ...","[2.6421990394592285, 2.8229875564575195, 2.558...",44941614
45337884,"[1475048, 2359003, 1469051, 1125186, 1045818, ...","[2.9149985313415527, 2.439664840698242, 3.1367...",45337884


In [170]:
ll = []
for i in dict.items():
    ll.append([x[0] for x in i[1]])
#print(ll)


In [172]:
rec_list = predict_filter.copy()
recl = rec_list.drop('book_id',axis=1)
recl = recl.drop('predicted_score',axis=1)

In [173]:
recl

,user_id
user_id,
1000152,1000152
1000176,1000176
1000182,1000182
1000288,1000288
1000430,1000430
...,...
44827605,44827605
44941614,44941614
45337884,45337884


In [174]:
recl['rec_list'] = ll

In [182]:
recl
recl.to_csv('./Recommend_list.csv',sep=',',index=False)

In [177]:
def getNDCG10(outputs, labels):
    dcgs_output = []
    idcgs_label = []
    for i in range(10):
        for items in labels:
            if items[0] == outputs[i][0]:
                dcgs_output.append(DCG(i+1,items[1]))
            if items[0] == labels[i][0]:
                idcgs_label.append(DCG(i+1,items[1]))
    return sum(dcgs_output)/sum(idcgs_label)

In [179]:
test_set

,user_id,book_id,book_score
184937,1487971,1959786,4
136250,1654151,1212894,5
86770,1115667,1006004,0
66072,3291995,1052380,0
138952,1746408,1424694,5
...,...,...,...
64196,3827505,1006073,0
98166,1120064,4113090,3
123254,1262387,1085860,5
125395,1799854,1071241,5


In [186]:
groundtruth = test_set.copy()

In [187]:
groundtruth = groundtruth.groupby(by='user_id').agg({'book_id':list,'book_score':list})
groundtruth['user_id'] = groundtruth.index
groundtruth.reset_index(drop=True)

,book_id,book_score,user_id
0,"[1054685, 2257188, 1019485, 2243692, 1007305, ...","[5, 4, 0, 3, 0, 4]",1000147
1,"[1084336, 1008145, 2036343, 1865891, 4886245, ...","[5, 5, 2, 0, 2, 4, 4, 4, 4, 3, 4, 4, 3, 5, 4, ...",1000152
2,"[1024197, 1003000, 1088022, 1291355, 1020670, ...","[0, 4, 4, 4, 4, 3, 3, 3, 4, 0, 5, 4, 5, 5, 3, ...",1000176
3,"[1089515, 1440589, 4238362, 1205054, 1044203, ...","[4, 4, 5, 3, 4, 5, 4, 5, 4, 4, 5, 5, 4, 3, 4, 3]",1000182
4,"[1079440, 1024217, 1059419, 1389960, 1002299, ...","[0, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4, 0]",1000288
...,...,...,...
2332,"[1085860, 1903968, 2057285, 2133074, 2265312, ...","[5, 4, 3, 5, 3, 5, 4, 3, 3, 4, 4, 5]",44827605
2333,"[1039487, 2265312, 3268399, 3646172, 1927902, ...","[0, 0, 5, 5, 0, 0, 5, 4, 3, 5, 0, 0, 5, 0, 4, ...",44941614
2334,"[1475048, 2359003, 1469051, 1125186, 1045818, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, ...",45337884
2335,"[1054889, 1057244, 1087145, 1026425, 1082154, ...","[0, 4, 0, 3, 4, 4, 5, 4, 4, 4, 0, 4, 4, 4, 5, 5]",47513621


In [188]:
lst = []
j = -1
for i in range(len(groundtruth['book_id'])):
    j += 1
    if len(groundtruth.iloc[i,0]) < 10:
        ls.append(groundtruth.iloc[i,2])
#print(ls)
ground = groundtruth.drop(ls,axis=0)
#print(ls)

In [189]:
ground

,book_id,book_score,user_id
user_id,,,
1000152,"[1084336, 1008145, 2036343, 1865891, 4886245, ...","[5, 5, 2, 0, 2, 4, 4, 4, 4, 3, 4, 4, 3, 5, 4, ...",1000152
1000176,"[1024197, 1003000, 1088022, 1291355, 1020670, ...","[0, 4, 4, 4, 4, 3, 3, 3, 4, 0, 5, 4, 5, 5, 3, ...",1000176
1000182,"[1089515, 1440589, 4238362, 1205054, 1044203, ...","[4, 4, 5, 3, 4, 5, 4, 5, 4, 4, 5, 5, 4, 3, 4, 3]",1000182
1000288,"[1079440, 1024217, 1059419, 1389960, 1002299, ...","[0, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4, 0]",1000288
1000430,"[1057601, 1024197, 4886245, 1051440, 1061563, ...","[0, 0, 5, 4, 0, 0, 4, 0, 0, 4]",1000430
...,...,...,...
44827605,"[1085860, 1903968, 2057285, 2133074, 2265312, ...","[5, 4, 3, 5, 3, 5, 4, 3, 3, 4, 4, 5]",44827605
44941614,"[1039487, 2265312, 3268399, 3646172, 1927902, ...","[0, 0, 5, 5, 0, 0, 5, 4, 3, 5, 0, 0, 5, 0, 4, ...",44941614
45337884,"[1475048, 2359003, 1469051, 1125186, 1045818, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, ...",45337884


In [219]:
dict1 = {}
for i in range(len(ground['user_id'])):
    dict1[str(ground.iloc[i,2])] = {}
    for j in range(len(ground.iloc[i, 0])):
        dict1[str(ground.iloc[i,2])][str(ground.iloc[i,0][j])] = ground.iloc[i,1][j]
dict1

{'1000152': {'1084336': 5,
  '1008145': 5,
  '2036343': 2,
  '1865891': 0,
  '4886245': 2,
  '1389960': 4,
  '1087145': 4,
  '1344548': 4,
  '1016523': 4,
  '1316648': 3,
  '1023045': 4,
  '1358413': 4,
  '3615061': 3,
  '1063624': 5,
  '1021847': 4,
  '1020961': 5,
  '2257188': 2,
  '2034893': 4,
  '1938591': 4,
  '3315879': 5,
  '1291355': 4,
  '1041616': 3,
  '4741216': 3,
  '1034282': 4,
  '2361768': 5,
  '1151876': 4,
  '1394364': 5},
 '1000176': {'1024197': 0,
  '1003000': 4,
  '1088022': 4,
  '1291355': 4,
  '1020670': 4,
  '1002299': 3,
  '1089521': 3,
  '1033778': 3,
  '1085470': 4,
  '1087758': 0,
  '1007928': 5,
  '1037602': 4,
  '2567698': 5,
  '1963786': 5,
  '1089523': 3,
  '1018136': 4,
  '1270508': 5},
 '1000182': {'1089515': 4,
  '1440589': 4,
  '4238362': 5,
  '1205054': 3,
  '1044203': 4,
  '1040771': 5,
  '2160292': 4,
  '3426869': 5,
  '1264579': 4,
  '1048537': 4,
  '1046265': 5,
  '1388169': 5,
  '1959786': 4,
  '2009825': 3,
  '1056868': 4,
  '1962929': 3},
 '10

In [220]:
for i in dict1.items():
    sorted_item = sorted(i[1].items(), key=lambda x: x[1], reverse=True)
    dict1[i[0]] = sorted_item[0:10]
    print(len(dict1[i[0]]))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

In [205]:
IDCG = []
temp = []
print(len(dict1.items()))
for i in dict1.items():
    temp = []
    for k in range(len(i)):
        #print(i[1])
        temp.append(DCG(k+1,i[1][k][1]))
    IDCG.append(sum(temp)/len(temp))
print(len(IDCG))

1806
1806


In [206]:
print(IDCG)

[0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.7337913755861883, 0.8154648767857288, 0.7337913755861883, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.4205487463657608, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.7337913755861883, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.7337913755861883, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.0, 0.0, 0.8154648767857288, 0.8154648767857288, 0.7337913755861883, 0.5, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.7337913755861883, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.8154648767857288, 0.81

In [226]:
dict1 = {}
for i in range(len(ground['user_id'])):
    dict1[str(ground.iloc[i,2])] = {}
    for j in range(len(ground.iloc[i, 0])):
        dict1[str(ground.iloc[i,2])][str(ground.iloc[i,0][j])] = ground.iloc[i,1][j]
dict1

{'1000152': {'1084336': 5,
  '1008145': 5,
  '2036343': 2,
  '1865891': 0,
  '4886245': 2,
  '1389960': 4,
  '1087145': 4,
  '1344548': 4,
  '1016523': 4,
  '1316648': 3,
  '1023045': 4,
  '1358413': 4,
  '3615061': 3,
  '1063624': 5,
  '1021847': 4,
  '1020961': 5,
  '2257188': 2,
  '2034893': 4,
  '1938591': 4,
  '3315879': 5,
  '1291355': 4,
  '1041616': 3,
  '4741216': 3,
  '1034282': 4,
  '2361768': 5,
  '1151876': 4,
  '1394364': 5},
 '1000176': {'1024197': 0,
  '1003000': 4,
  '1088022': 4,
  '1291355': 4,
  '1020670': 4,
  '1002299': 3,
  '1089521': 3,
  '1033778': 3,
  '1085470': 4,
  '1087758': 0,
  '1007928': 5,
  '1037602': 4,
  '2567698': 5,
  '1963786': 5,
  '1089523': 3,
  '1018136': 4,
  '1270508': 5},
 '1000182': {'1089515': 4,
  '1440589': 4,
  '4238362': 5,
  '1205054': 3,
  '1044203': 4,
  '1040771': 5,
  '2160292': 4,
  '3426869': 5,
  '1264579': 4,
  '1048537': 4,
  '1046265': 5,
  '1388169': 5,
  '1959786': 4,
  '2009825': 3,
  '1056868': 4,
  '1962929': 3},
 '10

In [227]:
for i in dict1.items():
    sorted_item = sorted(i[1].items(), key=lambda x: x[1], reverse=True)
    dict1[i[0]] = sorted_item
    print(len(dict1[i[0]]))

27
17
16
12
10
18
18
16
20
14
24
25
10
15
21
10
22
37
15
13
15
15
21
25
14
14
29
27
17
15
12
23
26
10
15
21
15
26
27
23
30
15
20
19
18
30
16
13
17
24
47
22
18
25
37
19
11
15
16
14
20
19
24
23
21
15
13
10
10
18
49
27
16
17
20
18
23
25
36
11
16
17
24
31
20
21
13
27
12
13
20
13
26
17
29
16
33
14
13
43
29
12
16
12
29
14
31
24
25
41
21
16
10
11
17
36
23
22
22
28
26
10
19
13
65
11
12
10
30
22
19
10
14
24
18
11
20
14
19
32
17
12
14
18
31
21
23
31
25
26
29
14
15
13
47
28
12
20
31
19
16
22
11
14
26
20
13
13
19
32
12
11
29
22
18
28
12
34
14
24
15
22
12
26
25
17
20
18
21
12
11
23
10
19
12
14
15
41
18
20
32
19
26
14
20
12
22
27
16
49
22
32
15
12
30
11
20
21
32
10
21
14
12
15
18
21
33
11
27
12
17
15
50
10
16
17
18
34
12
14
27
16
22
19
13
14
16
30
24
33
27
31
20
31
36
18
23
42
16
16
19
20
10
15
13
17
24
27
13
23
13
31
27
10
14
18
40
15
21
23
22
16
20
16
19
16
21
27
23
55
14
15
33
18
21
26
19
20
27
23
23
25
36
33
19
14
15
19
26
13
27
15
24
15
19
21
24
12
13
18
23
19
15
19
14
17
29
25
14
10
25
21
33
1

In [228]:
DCG1 = []
temp = []
print(len(dict.items()))
for i in dict.items():
    print(i[0])
    temp = []
    for k in range(len(i)):
        print(i[1][k])
        print(dict1[str(i[0])])
        for j in dict1[str(i[0])]:
            if j[0] == i[1][k][0]:
                temp.append(DCG(k+1,j[1]))
    DCG1.append(sum(temp)/len(temp))
print(DCG1)

1806
1000152
('1394364', 3.4075043201446533)
[('1084336', 5), ('1008145', 5), ('1063624', 5), ('1020961', 5), ('3315879', 5), ('2361768', 5), ('1394364', 5), ('1389960', 4), ('1087145', 4), ('1344548', 4), ('1016523', 4), ('1023045', 4), ('1358413', 4), ('1021847', 4), ('2034893', 4), ('1938591', 4), ('1291355', 4), ('1034282', 4), ('1151876', 4), ('1316648', 3), ('3615061', 3), ('1041616', 3), ('4741216', 3), ('2036343', 2), ('4886245', 2), ('2257188', 2), ('1865891', 0)]
('1316648', 3.392697811126709)
[('1084336', 5), ('1008145', 5), ('1063624', 5), ('1020961', 5), ('3315879', 5), ('2361768', 5), ('1394364', 5), ('1389960', 4), ('1087145', 4), ('1344548', 4), ('1016523', 4), ('1023045', 4), ('1358413', 4), ('1021847', 4), ('2034893', 4), ('1938591', 4), ('1291355', 4), ('1034282', 4), ('1151876', 4), ('1316648', 3), ('3615061', 3), ('1041616', 3), ('4741216', 3), ('2036343', 2), ('4886245', 2), ('2257188', 2), ('1865891', 0)]
1000176
('2567698', 3.379207134246826)
[('1007928', 5), ('

In [229]:
NDCG = []
for x in range(len(IDCG)):
    NDCG.append(DCG1[x]/IDCG[x])
print(NDCG)

[0.812653594257365, 0.812653594257365, 0.899844244032351, 0.8412568825598973, 0.31860741808177345, 0.812653594257365, 0.42991085379508587, 0.812653594257365, 0.899844244032351, 0.7030629722530329, 0.7411011265922482, 0.6029072162853839, 1.0, 0.812653594257365, 0.6029072162853839, 0.0, 0.8412568825598973, 0.7411011265922482, 0.8412568825598973, 0.7030629722530329, 0.0, 0.515716566510398, 0.2866970512668926, 0.8412568825598973, 0.8412568825598973, 1.0, 0.812653594257365, 0.6029072162853839, 0.6029072162853839, 0.8412568825598973, 0.0, 0.8412568825598973, 0.6539104768172622, 0.899844244032351, nan, nan, 0.899844244032351, 0.7411011265922482, 0.7266929595359539, 0.0, 0.7411011265922482, 0.6029072162853839, 0.6539104768172622, 0.8412568825598973, 0.8412568825598973, 0.515716566510398, 0.4544040753253556, 0.823588228192972, 0.6539104768172622, 0.4544040753253556, 0.6539104768172622, 0.899844244032351, 0.7411011265922482, 0.19950640149190665, 1.0, 0.899844244032351, 1.0, 0.6539104768172622, 0

<ipython-input-229-1c1857222cb0>:3: RuntimeWarning: invalid value encountered in double_scalars
  NDCG.append(DCG1[x]/IDCG[x])


In [230]:
recl['NDCG'] = NDCG

In [240]:
recl = recl.fillna(-1)

In [241]:
recl

,user_id,rec_list,NDCG
user_id,,,
1000152,1000152,"[1394364, 1316648, 1389960, 1023045, 2034893, ...",0.812654
1000176,1000176,"[2567698, 1089521, 1020670, 1018136, 1003000, ...",0.812654
1000182,1000182,"[4238362, 1959786, 1056868, 1440589, 1089515, ...",0.899844
1000288,1000288,"[1228110, 1389960, 1002299, 1059419, 1069685, ...",0.841257
1000430,1000430,"[1015767, 1051440, 1061563, 1027247, 4886245, ...",0.318607
...,...,...,...
44827605,44827605,"[1083428, 1085860, 2265312, 1040211, 1903968, ...",1.000000
44941614,44941614,"[3211779, 2209098, 3259440, 3646172, 1949338, ...",0.613147
45337884,45337884,"[1827374, 1007772, 1439216, 1469051, 1069685, ...",0.899844


In [242]:
recl.to_csv('./Final.csv',sep=',',index=False)

In [243]:
l = []
for i in recl['NDCG']:
    if i >= 0:
        l.append(i)
print(f"Average NDCG:{sum(l)/len(l)}")

Average NDCG:0.6510714408478413
